In [1]:
!unzip /content/drive/MyDrive/Colab\ Notebooks/Projects/TensorFlow/Competitions/Regression/Titanic-V2/data.zip

Archive:  /content/drive/MyDrive/Colab Notebooks/Projects/TensorFlow/Competitions/Regression/Titanic-V2/data.zip
replace gender_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [33]:
data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
sub = pd.read_csv('gender_submission.csv')

In [34]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [35]:
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [36]:
data.drop(['Cabin'],axis=1,inplace=True)

In [37]:
len(data)

891

In [38]:
data['Age'].fillna(data['Age'].mean(),inplace=True)

In [39]:
data.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

In [40]:
data.dropna(inplace=True)

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler

In [42]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Embarked        object
dtype: object

In [43]:
X = data.drop('Survived',axis=1)
y = data['Survived']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [44]:
ct = make_column_transformer(
    (MinMaxScaler(),['PassengerId','Pclass','Age','SibSp','Parch','Fare']),
    (OneHotEncoder(),['Name','Sex','Ticket','Embarked'])
)
ct.fit(X)
X_train,X_test = ct.transform(X_train),ct.transform(X_test)
X_train,X_test = X_train.toarray(),X_test.toarray()

# Modelling

In [45]:
!pip3 install wandb

In [46]:
import wandb
from wandb.keras import *

In [47]:
wandb.init(project='titanic-v2',name='baseline')
baseline = tf.keras.Sequential([
  tf.keras.layers.Dense(1)
])
baseline.compile(loss=tf.keras.losses.mae,metrics=['mse'],optimizer=tf.keras.optimizers.Adam())
baseline.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[WandbCallback()])

epoch,99
loss,0.7864
mse,4.20093
val_loss,2.42359
val_mse,1.06454
_runtime,285
_timestamp,1616649516
_step,99
best_val_loss,1.7929
best_epoch,3


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,▁█▇▅▅▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
val_loss,▇▁▁█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
val_mse,▄█▆▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


Epoch 1/100
21/21 [==============================] - 1s 15ms/step - loss: 0.4209 - mse: 0.3919 - val_loss: 0.3977 - val_mse: 0.3422
Epoch 2/100
21/21 [==============================] - 0s 4ms/step - loss: 0.3735 - mse: 0.3202 - val_loss: 0.3907 - val_mse: 0.3294
Epoch 3/100
21/21 [==============================] - 0s 4ms/step - loss: 0.4038 - mse: 0.3426 - val_loss: 0.3842 - val_mse: 0.3157
Epoch 4/100
21/21 [==============================] - 0s 4ms/step - loss: 0.3144 - mse: 0.2549 - val_loss: 0.3790 - val_mse: 0.2989
Epoch 5/100
21/21 [==============================] - 0s 4ms/step - loss: 0.3567 - mse: 0.2804 - val_loss: 0.3726 - val_mse: 0.2878
Epoch 6/100
21/21 [==============================] - 0s 4ms/step - loss: 0.3337 - mse: 0.2555 - val_loss: 0.3674 - val_mse: 0.2754
Epoch 7/100
21/21 [==============================] - 0s 4ms/step - loss: 0.3167 - mse: 0.2348 - val_loss: 0.3619 - val_mse: 0.2615
Epoch 8/100
21/21 [==============================] - 0s 5ms/step - loss: 0.3038 - 

In [76]:
class Experiment:
  def experiment(self,amount_of_layers:int,epochs:int,lr:int,optimizer_,neurons_per_layer:int,activation_types:list,metrics_:list,name:str,project_name:str):
    wandb.init(project=project_name,name=name)
    model = tf.keras.Sequential()
    if int(amount_of_layers) != len(activation_types):
      activation_types_new = []
      for _ in range(amount_of_layers):
        activation_types_new.append(activation_types[0])
    for activation_type in activation_types:
      print(activation_type)
      model.add(tf.keras.layers.Dense(neurons_per_layer,activation=activation_type))
    # model.add(tf.keras.layers.Dense(1))
    model.compile(loss=tf.keras.losses.binary_crossentropy,metrics=metrics_,optimizer=optimizer_(lr=lr))
    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=epochs,callbacks=[WandbCallback()],verbose=0)
    return model

In [77]:
# Layers = 1,2,3,4,5,10,15,20,25
# Epochs = 125,250,500
# learning rate = 1.0,0.1,0.01,0.001
# total neurons per layer = 25000,50000,75000,100000
# activation type = linear Swish ReLU Tanh selu elu
# optimize = SGD, Adam, Adadelta, Adagrad, Adamax

In [78]:
E = Experiment()

In [79]:
# Layers

In [80]:
from tensorflow.keras.optimizers import *

In [81]:
# layers = [1,2,3,4,5,10,15,20,25]
# for layer in layers:
# E.experiment(amount_of_layers=layer,
#               epochs=100,
#               lr=0.001,
#               optimizer_=Adam,
#               neurons_per_layer=1,
#               activation_types=[None],
#               metrics_=['mse'],
#               name=f'{layer}-layer',
#               project_name='titanic-v2')
# print('Done') 

In [82]:
# Epochs

In [83]:
# epochs = [125,250,500]
# for epoch in epochs:
# E.experiment(amount_of_layers=1,
#               epochs=epoch,
#               lr=0.001,
#               optimizer_=Adam,
#               neurons_per_layer=1,
#               activation_types=[None],
#               metrics_=['mse'],
#               name=f'{epoch}-epoch',
#               project_name='titanic-v2')
# print('Done') 

In [84]:
# lr

In [85]:
# lrs = [1.0,10.0,0.1,0.01,0.001]
# for lr in lrs:
#   E.experiment(amount_of_layers=1,
#                 epochs=100, 
#                 lr=lr, 
#                 optimizer_=Adam,
#                 neurons_per_layer=1,
#                 activation_types=[None],
#                 metrics_=['mse'],
#                 name=f'{lr}-lr',
#                 project_name='titanic-v2')
# print('Done') 

In [86]:
# Neurons per layer

In [87]:
# neurons = [1250,2500,5000,7500,10000,12500]
# for neuron in neurons:
#   E.experiment(
#       amount_of_layers=1,
#       epochs=100,
#       lr=0.001,
#       optimizer_=tf.keras.optimizers.Adam,
#       neurons_per_layer=neuron,
#       activation_types=[None],
#       metrics_=['mse'],
#       name=f'{neuron}-neuron',
#       project_name='titanic-v2'
#   )
# print('Done')

In [88]:
# activation types

In [89]:
# ats = ['swish','relu','tanh','selu','elu']
# for at in ats:
#   E.experiment(
#       amount_of_layers=1,
#       epochs=100,
#       lr=0.001,
#       optimizer_=tf.keras.optimizers.Adam,
#       neurons_per_layer=1,
#       activation_types=[at],
#       metrics_=['mse'],
#       name=f'{at}-at',
#       project_name='titanic-v2'
#   )
# print('Done')

epoch,111
loss,0.0
mse,0.04242
val_loss,0.75729
val_mse,0.23452
_runtime,18
_timestamp,1616652970
_step,111
best_val_loss,0.51716
best_epoch,5


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▄▂▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
val_loss,▅▃▁▁▃▃▃▃▅▅▆▆▇▇██████████████████████████
val_mse,█▄▁▂▄▄▅▅▅▆▆▇▇▇▇▇█▇██████████████████████
_runtime,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇█
_timestamp,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇█
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


swish


epoch,99
loss,0.22968
mse,0.07729
val_loss,0.67759
val_mse,0.23949
_runtime,14
_timestamp,1616652990
_step,99
best_val_loss,0.67759
best_epoch,99


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
val_loss,█▄▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mse,█▇▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
_runtime,▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇████
_timestamp,▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▆▆▅▅▅▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mse,█▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


selu


epoch,99
loss,0.15743
mse,0.05039
val_loss,0.67777
val_mse,0.2351
_runtime,14
_timestamp,1616653049
_step,99
best_val_loss,0.67777
best_epoch,99


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▇▇▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_loss,█▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mse,█▇▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
_runtime,▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇██
_timestamp,▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇██
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


elu
Done


In [90]:
# optimizer

In [91]:
# optimizers = [SGD,Adam,Adadelta,Adagrad,Adamax]
# for optimizer in optimizers:
#   E.experiment(
#       amount_of_layers=1,
#       epochs=100,
#       lr=0.001,
#       optimizer_=optimizer,
#       neurons_per_layer=1,
#       activation_types=[None],
#       metrics_=['mse'],
#       name=f'{optimizer}-optimizer',
#       project_name='titanic-v2'
#   )
# print('Done')

epoch,99
loss,0.03763
mse,0.01229
val_loss,0.61307
val_mse,0.20846
_runtime,13
_timestamp,1616653066
_step,99
best_val_loss,0.54904
best_epoch,53


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mse,█▆▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂
_runtime,▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇████
_timestamp,▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


None


epoch,99
loss,0.43277
mse,0.14109
val_loss,0.4845
val_mse,0.15895
_runtime,13
_timestamp,1616653091
_step,99
best_val_loss,0.4845
best_epoch,99


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mse,█▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇███
_timestamp,▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█████▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁
mse,████▇▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▁▁
val_loss,███████▆▆▆▆▆▆▅▅▅▅▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_mse,████▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▁▁
_runtime,▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇█
_timestamp,▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇█
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


None


epoch,99
loss,1.56192
mse,0.32816
val_loss,2.35021
val_mse,0.35158
_runtime,14
_timestamp,1616653146
_step,99
best_val_loss,2.35021
best_epoch,99


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▅▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▆▅▅▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_loss,██▆▅▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mse,█▇▆▅▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇██
_timestamp,▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇██
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


None
Done


In [92]:
# Results
# Layers = 5
# Epochs = 125 (With early stopping)
# learning rate = 0.01
# total neurons per layer = 2500
# activation type = swish (None Swish ReLU Tanh selu elu)
# optimize = SGD (SGD, Adam, Adadelta, Adagrad, Adamax)

In [93]:
model = E.experiment(
    amount_of_layers=5,
    epochs=112,
    lr=0.01,
    optimizer_=tf.keras.optimizers.SGD,
    neurons_per_layer=2500,
    activation_types=['swish'],
    metrics_=['mse'],
    name=f'Big-Dog-Model-1',
    project_name='titanic-v2'
)
print(model.save('/content/drive/MyDrive/Colab Notebooks/Projects/TensorFlow/Competitions/Regression/Titanic-V2/model_1.h5'))
print('Done')

epoch,99
loss,0.51822
mse,0.19515
val_loss,0.70992
val_mse,0.25268
_runtime,13
_timestamp,1616653166
_step,99
best_val_loss,0.70992
best_epoch,99


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▇▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁
val_loss,█▅▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mse,█▇▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
_runtime,▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇███
_timestamp,▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


swish
None
Done


In [94]:
model = E.experiment(
    amount_of_layers=1,
    epochs=112,
    lr=0.001,
    optimizer_=tf.keras.optimizers.SGD,
    neurons_per_layer=2500,
    activation_types=['swish'],
    metrics_=['mse'],
    name=f'Big-Dog-Model-1',
    project_name='titanic-v2'
)
print(model.save('/content/drive/MyDrive/Colab Notebooks/Projects/TensorFlow/Competitions/Regression/Titanic-V2/model_1.h5'))
print('Done')

epoch,111
loss,0.99513
mse,0.32099
val_loss,1.00111
val_mse,0.32369
_runtime,77
_timestamp,1616653247
_step,111
best_val_loss,1.00111
best_epoch,111


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▇▆▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
val_loss,█▇▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mse,█▇▆▆▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


swish
None
Done


In [ ]:
model = E.experiment(
    amount_of_layers=5,
    epochs=112,
    lr=0.001,
    optimizer_=tf.keras.optimizers.SGD,
    neurons_per_layer=2500,
    activation_types=[None],
    metrics_=['mse'],
    name=f'Big-Dog-Model-1',
    project_name='titanic-v2'
)
print(model.save('/content/drive/MyDrive/Colab Notebooks/Projects/TensorFlow/Competitions/Regression/Titanic-V2/model_1.h5'))
print('Done')

epoch,111
loss,1.76633
mse,0.35777
val_loss,1.76594
val_mse,0.36131
_runtime,78
_timestamp,1616653335
_step,111
best_val_loss,1.76594
best_epoch,111


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
mse,███▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
val_loss,███▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
val_mse,███▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


None


In [ ]:
model = E.experiment(
    amount_of_layers=1,
    epochs=112,
    lr=0.001,
    optimizer_=tf.keras.optimizers.SGD,
    neurons_per_layer=2500,
    activation_types=[None],
    metrics_=['mse'],
    name=f'Big-Dog-Model-1',
    project_name='titanic-v2'
)
print(model.save('/content/drive/MyDrive/Colab Notebooks/Projects/TensorFlow/Competitions/Regression/Titanic-V2/model_1.h5'))
print('Done')

epoch,111
loss,1.56783
mse,0.33871
val_loss,1.56199
val_mse,0.34172
_runtime,78
_timestamp,1616653417
_step,111
best_val_loss,1.56199
best_epoch,111


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
mse,███▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
val_loss,██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
val_mse,███▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


None
None
Done


In [ ]:
model = E.experiment(
    amount_of_layers=5,
    epochs=112,
    lr=0.01,
    optimizer_=tf.keras.optimizers.Adam,
    neurons_per_layer=2500,
    activation_types=['swish'],
    metrics_=['mse'],
    name=f'Big-Dog-Model-1',
    project_name='titanic-v2'
)
print(model.save('/content/drive/MyDrive/Colab Notebooks/Projects/TensorFlow/Competitions/Regression/Titanic-V2/model_2.h5'))
print('Done')

epoch,111
loss,1.56307
mse,0.33878
val_loss,1.55865
val_mse,0.34181
_runtime,76
_timestamp,1616653503
_step,111
best_val_loss,1.55865
best_epoch,111


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
mse,███▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
val_loss,██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
val_mse,███▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


swish
None
Done


In [73]:
model = E.experiment(
    amount_of_layers=5,
    epochs=112,
    lr=0.001,
    optimizer_=tf.keras.optimizers.Adam,
    neurons_per_layer=2500,
    activation_types=['swish'],
    metrics_=['mse'],
    name=f'Big-Dog-Model-1',
    project_name='titanic-v2'
)
print(model.save('/content/drive/MyDrive/Colab Notebooks/Projects/TensorFlow/Competitions/Regression/Titanic-V2/model_2.h5'))
print('Done')

epoch,111
loss,9.43346
mse,101.32533
val_loss,9.36837
val_mse,103.98442
_runtime,76
_timestamp,1616652829
_step,111
best_val_loss,9.36837
best_epoch,0


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▁███████████████████████████████████████
mse,▁███████████████████████████████████████
val_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mse,▁███████████████████████████████████████
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


swish
None
Done


In [74]:
model = E.experiment(
    amount_of_layers=5,
    epochs=112,
    lr=0.01,
    optimizer_=tf.keras.optimizers.Adam,
    neurons_per_layer=100,
    activation_types=['swish'],
    metrics_=['mse'],
    name=f'Big-Dog-Model-1',
    project_name='titanic-v2'
)
print(model.save('/content/drive/MyDrive/Colab Notebooks/Projects/TensorFlow/Competitions/Regression/Titanic-V2/model_2.h5'))
print('Done')

epoch,111
loss,0.0
mse,0.0731
val_loss,0.5135
val_mse,0.17125
_runtime,79
_timestamp,1616652919
_step,111
best_val_loss,0.46704
best_epoch,1


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
val_loss,▁▁█▆█▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
val_mse,▂▁▇▆█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


swish
None
Done


In [75]:
model = E.experiment(
    amount_of_layers=5,
    epochs=112,
    lr=0.001,
    optimizer_=tf.keras.optimizers.Adam,
    neurons_per_layer=100,
    activation_types=['swish'],
    metrics_=['mse'],
    name=f'Big-Dog-Model-1',
    project_name='titanic-v2'
)
print(model.save('/content/drive/MyDrive/Colab Notebooks/Projects/TensorFlow/Competitions/Regression/Titanic-V2/model_2.h5'))
print('Done')

epoch,111
loss,0.02316
mse,0.43664
val_loss,0.69065
val_mse,0.18316
_runtime,18
_timestamp,1616652942
_step,111
best_val_loss,0.62055
best_epoch,1


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,▁▂██████████████████████████████████████
val_loss,▁█▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
val_mse,▁█▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
_runtime,▁▁▁▁▁▁▂▂▃▃▃▃▃▃▃▃▃▄▄▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇█
_timestamp,▁▁▁▁▁▁▂▂▃▃▃▃▃▃▃▃▃▄▄▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇█
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


swish
None
Done


In [ ]:
model = E.experiment(
    amount_of_layers=1,
    epochs=112,
    lr=0.001,
    optimizer_=tf.keras.optimizers.Adam,
    neurons_per_layer=100,
    activation_types=['swish'],
    metrics_=['mse'],
    name=f'Big-Dog-Model-1',
    project_name='titanic-v2'
)
print(model.save('/content/drive/MyDrive/Colab Notebooks/Projects/TensorFlow/Competitions/Regression/Titanic-V2/model_2.h5'))
print('Done')

epoch,111
loss,0.17196
mse,0.03033
val_loss,0.73822
val_mse,0.20145
_runtime,85
_timestamp,1616653592
_step,111
best_val_loss,0.67628
best_epoch,7


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
val_mse,█▄▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
_runtime,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


swish
None
Done
